In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os


DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SEED = 42


def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [18]:
df = pd.read_csv(f"{DATA_PATH}train.csv")
df_test = pd.read_csv(f"{DATA_PATH}test.csv")
df_submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

# 데이터

In [14]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 47.1 MB/s eta 0:00:00


In [15]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

def calculate_2d_3d_combined_descriptor(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES string")

    # Calculate 2D descriptors
    mw = Descriptors.MolWt(mol)  # Molecular weight
    logp = Descriptors.MolLogP(mol)  # LogP

    # Generate 3D coordinates
    mol = Chem.AddHs(mol)  # Add hydrogens for accurate 3D calculation
    AllChem.EmbedMolecule(mol, randomSeed=42)  # Embed the molecule in 3D space

    # Calculate 3D descriptor: Predicted molecular volume
    volume_3d = AllChem.ComputeMolVolume(mol)

    # Combine 2D and 3D descriptors into a dictionary
    combined_descriptor = {
        'MolecularWeight': mw,
        'LogP': logp,
        'Volume3D': volume_3d
    }

    return combined_descriptor

In [21]:
# 기존 데이터프레임에서 SMILES 열 추출
smiles_list = df['SMILES'].tolist()

# LogP 계산 및 tqdm 적용
logP_values = []
for smiles in tqdm(smiles_list, desc="Calculating LogP"):
    descriptor = calculate_2d_3d_combined_descriptor(smiles)
    logP_values.append(descriptor['LogP'])

# 계산 결과를 데이터프레임에 추가
df['LogP'] = logP_values

Calculating LogP:   0%|          | 0/3498 [00:00<?, ?it/s]

[22:11:14] UFFTYPER: Unrecognized atom type: Se2+2 (8)
[22:11:14] UFFTYPER: Unrecognized atom type: Se2+2 (8)
[22:12:54] UFFTYPER: Unrecognized atom type: Se2+2 (6)


In [46]:
# 기존 데이터프레임에서 SMILES 열 추출
smiles_list = df_test['SMILES'].tolist()

# LogP 계산 및 tqdm 적용
logP_values = []
for smiles in tqdm(smiles_list, desc="Calculating LogP"):
    descriptor = calculate_2d_3d_combined_descriptor(smiles)
    logP_values.append(descriptor['LogP'])

# 계산 결과를 데이터프레임에 추가
df_test['LogP'] = logP_values

Calculating LogP:   0%|          | 0/483 [00:00<?, ?it/s]

In [48]:
df_test['LogP']

0      2.43160
1      1.82520
2      3.27051
3      2.03830
4      1.27232
        ...   
478    3.81860
479    0.01480
480    2.32600
481    2.24480
482    2.65670
Name: LogP, Length: 483, dtype: float64

In [35]:
# 기존 데이터프레임에서 SMILES 열 추출
smiles_list = df['SMILES'].tolist()

# Volume 3d 계산 및 tqdm 적용
Volume3d_values = []
for smiles in tqdm(smiles_list, desc="Calculating Volume3d"):
    descriptor = calculate_2d_3d_combined_descriptor(smiles)
    Volume3d_values.append(descriptor['Volume3D'])

# 계산 결과를 데이터프레임에 추가
df['Volume3d'] = Volume3d_values


Calculating Volume3d:   0%|          | 0/3498 [00:00<?, ?it/s]

[00:24:12] UFFTYPER: Unrecognized atom type: Se2+2 (8)
[00:24:12] UFFTYPER: Unrecognized atom type: Se2+2 (8)
[00:25:52] UFFTYPER: Unrecognized atom type: Se2+2 (6)


In [47]:
# 기존 데이터프레임에서 SMILES 열 추출
smiles_list = df_test['SMILES'].tolist()

# Volume 3d 계산 및 tqdm 적용
Volume3d_values = []
for smiles in tqdm(smiles_list, desc="Calculating Volume3d"):
    descriptor = calculate_2d_3d_combined_descriptor(smiles)
    Volume3d_values.append(descriptor['Volume3D'])

# 계산 결과를 데이터프레임에 추가
df_test['Volume3d'] = Volume3d_values

Calculating Volume3d:   0%|          | 0/483 [00:00<?, ?it/s]

In [50]:
df_test['Volume3d'] = Volume3d_values
df_test

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,LogP,Volume3d
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,2.43160,337.504
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,1.82520,327.936
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,3.27051,322.296
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,2.03830,294.400
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,1.27232,320.632
...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13,3.81860,315.128
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16,0.01480,309.800
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72,2.32600,312.832
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64,2.24480,239.368


In [51]:
#분자량 500이하
df['lip_MolWeight'] = (df['Molecular_Weight'] <= 500).astype(int)
df['lip_LogP'] = (df['LogP'] <= 5).astype(int)
df['lip_H_Acc'] = (df['Num_H_Acceptors'] <= 10).astype(int)
df['lip_H_Don'] = (df['Num_H_Donors'] <= 5).astype(int)
df['lip_RB'] = (df['Num_RotatableBonds'] <= 5).astype(int)
df['lip_pass'] = df['lip_MolWeight']*df['lip_LogP']*df['lip_H_Acc']*df['lip_H_Don']*df['lip_RB']

df_test['lip_MolWeight'] = (df_test['Molecular_Weight'] <= 500).astype(int)
df_test['lip_LogP'] = (df_test['LogP'] <= 5).astype(int)
df_test['lip_H_Acc'] = (df_test['Num_H_Acceptors'] <= 10).astype(int)
df_test['lip_H_Don'] = (df_test['Num_H_Donors'] <= 5).astype(int)
df_test['lip_RB'] = (df_test['Num_RotatableBonds'] <= 5).astype(int)
df_test['lip_pass'] = df_test['lip_MolWeight']*df_test['lip_LogP']*df_test['lip_H_Acc']*df_test['lip_H_Don']*df_test['lip_RB']


In [52]:
df_test

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,LogP,Volume3d,lip_MolWeight,lip_LogP,lip_H_Acc,lip_H_Don,lip_RB,lip_pass
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,2.43160,337.504,1,1,1,1,0,0
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,1.82520,327.936,1,1,1,1,1,1
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,3.27051,322.296,1,1,1,1,1,1
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,2.03830,294.400,1,1,1,1,1,1
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,1.27232,320.632,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13,3.81860,315.128,1,1,1,1,0,0
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16,0.01480,309.800,1,1,1,1,1,1
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72,2.32600,312.832,1,1,1,1,1,1
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64,2.24480,239.368,1,1,1,1,1,1


In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [54]:
# seed
seed = 42

# define rmse
rmse = lambda x, y: np.mean((x - y) ** 2) ** 0.5

# features and targets
features = ["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea",'LogP','Volume3d','lip_pass']
mlm_target = "MLM"
hlm_target = "HLM"


In [56]:
df[features]

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,LogP,Volume3d,lip_pass
0,3.259,400.495,5,2,8,3.259,117.37,3.87744,361.640,0
1,2.169,301.407,2,1,2,2.172,73.47,3.35474,273.504,1
2,1.593,297.358,5,0,3,1.585,62.45,1.20450,269.576,1
3,4.771,494.652,6,0,5,3.475,92.60,3.89356,451.184,1
4,2.335,268.310,3,0,1,2.337,42.43,2.81772,246.040,1
...,...,...,...,...,...,...,...,...,...,...
3493,3.409,396.195,3,1,5,3.409,64.74,2.74730,304.304,1
3494,1.912,359.381,4,1,3,1.844,77.37,2.27630,313.496,1
3495,1.941,261.320,3,1,6,2.124,70.14,2.04130,249.664,0
3496,0.989,284.696,5,1,5,0.989,91.51,1.42720,236.384,1


In [61]:
df_test[features]

KeyError: ignored

In [55]:
df["AlogP"] = np.where(pd.isna(df["AlogP"]), df["LogD"], df["AlogP"])


In [39]:
df

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,LogP,Volume3d,lip_MolWeight,lip_LogP,lip_H_Acc,lip_H_Don,lip_RB,lip_pass
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,3.87744,361.640,1,1,1,1,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,3.35474,273.504,1,1,1,1,1,1
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,1.20450,269.576,1,1,1,1,1,1
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,3.89356,451.184,1,1,1,1,1,1
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,2.81772,246.040,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74,2.74730,304.304,1,1,1,1,1,1
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37,2.27630,313.496,1,1,1,1,1,1
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14,2.04130,249.664,1,1,1,1,0,0
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51,1.42720,236.384,1,1,1,1,1,1


In [58]:

# train
scores = []
reg_mlms = []
reg_hlms = []
kf = KFold(n_splits = 10, random_state = seed, shuffle = True)
for i, (train_index, valid_index) in enumerate(kf.split(df)):
    df_train = df.iloc[train_index]
    df_valid = df.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    reg_mlm = RandomForestRegressor(random_state = seed)
    reg_mlm.fit(x_train, y_mlm_train)
    p_mlm = reg_mlm.predict(x_valid)

    reg_hlm = RandomForestRegressor(random_state = seed)
    reg_hlm.fit(x_train, y_hlm_train)
    p_hlm = reg_hlm.predict(x_valid)

    score = 0.5 * rmse(y_mlm_valid, p_mlm) + 0.5 * rmse(y_hlm_valid, p_hlm)

    reg_mlms.append(reg_mlm)
    reg_hlms.append(reg_hlm)
    scores.append(score)
    print(f"Fold {i+1:2d}: {score:.5f}")

print(f"CV score: {np.mean(scores):.5f}")

Fold  1: 33.97490
Fold  2: 33.08365
Fold  3: 32.59282
Fold  4: 34.30054
Fold  5: 32.01334
Fold  6: 32.96770
Fold  7: 31.20206
Fold  8: 32.15057
Fold  9: 31.88619
Fold 10: 32.59398
CV score: 32.67657


In [60]:
# load data
df_test = pd.read_csv(f"{DATA_PATH}test.csv")
df_test["AlogP"] = np.where(pd.isna(df_test["AlogP"]), df_test["LogD"], df_test["AlogP"])

# predict
df_submission =  pd.read_csv(f"{DATA_PATH}sample_submission.csv")
df_submission["MLM"] = np.mean([reg_mlm.predict(df_test[features].values) for reg_mlm in reg_mlms], axis = 0)
df_submission["HLM"] = np.mean([reg_hlm.predict(df_test[features].values) for reg_hlm in reg_hlms], axis = 0)
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

KeyError: ignored